In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
from scipy import sparse
import itertools
%matplotlib inline

!pip install pymorphy2
!pip install textblob
!pip install pyaspeller
!pip install deep_translator
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

from multiprocessing import Pool
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

from textblob import TextBlob
from pyaspeller import YandexSpeller
from deep_translator import GoogleTranslator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score

from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegressionCV

import pickle

np.random.seed(42)
random.seed(42)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrakozevnikova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_df = pd.read_csv('train_ml.csv')

In [5]:
test_df = pd.read_csv('new_test_ml.csv')

In [6]:
train_df.head(2)

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,13.12.2016 1:05


In [7]:
train_df['date'] = pd.to_datetime(train_df.date)

In [8]:
train_df.date.max()

Timestamp('2021-12-03 23:36:00')

In [9]:
train_df.date.min()

Timestamp('2005-01-07 16:19:00')

In [10]:
train_df.grades.value_counts()# / train_df[train_df.grades >= 1].shape[0]

1.0    27739
5.0    14227
2.0     5634
3.0     2356
4.0     1520
Name: grades, dtype: int64

In [11]:
train_df[(train_df.grades.isna())]

,bank,feeds,grades,date
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00
5,pochtabank,Брала кредит на стиральную машину. Все платила...,NaN,2015-09-04 17:19:00
6,otpbank,"Откуда взялся долг по кредитной карте, если я ...",NaN,2021-01-28 13:20:00
9,fk_otkritie,"Уважаемый Бинбанк, если у вас имеются какие-ли...",NaN,2018-08-31 16:48:00
...,...,...,...,...
74981,vtb,13 августа 2018 г. в ДО ЦИК «Каретный Ряд» под...,NaN,2018-05-09 14:09:00
74984,vozrozhdenie,Обман со стороны банка. При снятии наличности ...,NaN,2013-10-02 21:46:00
74985,v-express-bank,Здравствуйте! По действующему ипотечному креди...,NaN,2020-06-23 11:26:00
74988,rsb,"Впервые сталкиваюсь с такой ситуацией, когда б...",NaN,2015-06-18 13:34:00


### Тестовые

In [12]:
test_df.drop(columns=['Unnamed: 0'], inplace=True)

In [13]:
test_df.shape

(17220, 3)

In [14]:
test_df.head()

,bank,feeds,date
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53
1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48
4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41


# Начнем с предобработки

In [15]:
stop_words_our = ['а', 'более', 
                 'больше', 'будет',
                 'будто', 'бы',
                 'был', 'была',
                 'были', 'было',
                 'быть', 'вдруг',
                 'во', 'впрочем',
                 'все', 'всегда',
                 'всего', 'всех',
                 'всю', 'г', 'где',
                 'для', 'до', 'другой',
                 'если','есть',
                 'еще', 'ж',
                 'же', 'за',
                 'зачем','из',
                 'или', 'иногда',
                 'к', 'когда',
                 'куда', 'между',
                 'много', 'над',
                 'нибудь', 'о',
                 'об', 'от', 'перед',
                 'по',  'под',
                 'после', 'потом',
                 'почти', 'при',
                 'про', 'разве', 'с',
                 'сейчас', 'со',
                 'тем', 'то', 'только',
                 'том', 'у', 'уж',
                 'уже', 'хоть',
                 'чем', 'чтобы',
                 'чуть', 'я', 'и']

**Исправление орфографических ошибок**

In [16]:
speller = YandexSpeller()
text = test_df.feeds[2]
text

'Добрый день, уважаемые сотрудники службы контроля качества данного " замечательного " банка. Когда же решится сой вопрос с моей кредитной картой? Ранее была выдана мне информация неким сотрудником службы контроля качества Вадимом М., что карта моя выпущена 30.01.2015 года и направлена мне. Далее поступила информация от некого сотрудника также службы контроля качества Козырева Эльдара, который мне сообщил, что карта моя перевыпущена уже 13.01.2016 года и в течении месяца должна быть доставлена мне. Сегодня уже 20. 02.2016 год. Вам самим не смешно без конца просто отписываться мне и кормить завтраками? Когда будет решён мой вопрос? Я воспользуюсь возможность и оставлю претензию на сайте ЦБ Росии о деятельности вашего банка.'

In [17]:
str(TextBlob(text).correct())

'Добрый день, уважаемые сотрудники службы контроля качества данного " замечательного " банка. Когда of решится сой вопрос с моей кредитной картой? Ранее была выдана мне информация неким сотрудником службы контроля качества Вадимом М., что карта моя выпущена 30.01.2015 года и направлена мне. Далее поступила информация of некого сотрудника также службы контроля качества Козырева Эльдара, который мне сообщил, что карта моя перевыпущена уже 13.01.2016 года и в течении месяца должна быть доставлена мне. Сегодня уже 20. 02.2016 год. Вам самим of смешно без конца просто отписываться мне и кормить завтраками? Когда будет решён мой вопрос? Я воспользуюсь возможность и оставлю претензию of сайте of Росии о деятельности вашего банка.'

In [18]:
fixed = speller.spelled(text)
fixed

'Добрый день, уважаемые сотрудники службы контроля качества данного " замечательного " банка. Когда же решится свой вопрос с моей кредитной картой? Ранее была выдана мне информация неким сотрудником службы контроля качества Вадимом М., что карта моя выпущена 30.01.2015 года и направлена мне. Далее поступила информация от некого сотрудника также службы контроля качества Козырева Эльдара, который мне сообщил, что карта моя перевыпущена уже 13.01.2016 года и в течении месяца должна быть доставлена мне. Сегодня уже 20. 02.2016 год. Вам самим не смешно без конца просто отписываться мне и кормить завтраками? Когда будет решён мой вопрос? Я воспользуюсь возможность и оставлю претензию на сайте ЦБ России о деятельности вашего банка.'

Вывод:
Yandex Speller работает лучше

In [19]:
POINTS = {'?' : 'вопрос',
          '!' : 'восклицание'}

In [20]:
def prep_text_new(texts):
    speller = YandexSpeller()
    stop_words = stop_words_our
    regex = re.compile('[^a-z A-z а-я А-Я !?]')
    regex_1 = re.compile('[^a-z а-я]')
    
    preprocess_texts = []
    caps_num = []
    caps_words = []
    for i in tqdm(range(len(texts))):     
        # Поиск CAPS
        caps_counter = 0
        upper_words = []
        for word in texts[i].split():
            if word.isupper():
                caps_counter += 1
                word = regex_1.sub(' ', word.lower())
                if not word in stop_words:
                    upper_words.append(word)
        caps_num.append(caps_counter)
        caps_words.append(' '.join(upper_words))
        
        # Регистр
        text = texts[i].lower()
        text = text.replace('ё', 'е')
        text = text.replace('-', '')
        
        #Удаление веб-адресов:
        text = ' '.join(re.sub("(\w+:\/\/\S+)", "сайт", text).split())
        text = ' '.join(re.sub(r'httpS+', "сайт", text).split())
        
        #Удаление хэштегов / аккаунтов
        text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", text).split())
        
        # исправление слов с ошибками
        text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
        
        # Иставляем только нужные символы
        text = regex.sub(' ', text)
        
        # исправление орфографических ошибок
        try:
            text = speller.spelled(text)
        except:
            text = str(TextBlob(text).correct())
        
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        
        # заменим знаки препинания
        prep = ' '.join([POINTS[word] if word in POINTS else word for word in filtered_sentence])
                
        preprocess_texts.append(prep)
    return preprocess_texts, caps_num, caps_words

In [22]:
# запустим предобработку
train_df['prep_feeds'], train_df['cups_num'], train_df['cups_words'] = prep_text_new(train_df.feeds.to_list())

100%|██████████| 75000/75000 [5:08:08<00:00,  4.06it/s]    


In [23]:
train_df.to_csv('prep_train.csv', index = False)

In [24]:
test_df['prep_feeds'], test_df['cups_num'], test_df['cups_words'] = prep_text_new(test_df.feeds.to_list())

100%|██████████| 17220/17220 [1:23:03<00:00,  3.46it/s]


In [25]:
test_df.to_csv('prep_test.csv', index = False)

In [28]:
train_df = pd.read_csv('prep_train.csv')

In [29]:
test_df = pd.read_csv('prep_test.csv')

In [30]:
train_df.head(2)

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао


## **Стемминг**

In [31]:
def stemming_texts(texts):
    st = SnowballStemmer("russian")
    stem_text = []
    for text in tqdm(texts):
        word_tokens = word_tokenize(text)
        stem_text.append(' '.join([st.stem(word) for word in word_tokens]))
    return stem_text

In [32]:
train_df['stem_text'] = stemming_texts(train_df.prep_feeds.tolist())

100%|██████████| 75000/75000 [10:31<00:00, 118.78it/s]


In [33]:
train_df.to_csv('stem_prep_train.csv', index=False)

In [34]:
test_df['stem_text'] = stemming_texts(test_df.prep_feeds.tolist())

100%|██████████| 17220/17220 [02:24<00:00, 118.77it/s]


In [35]:
test_df.to_csv('stem_prep_test.csv', index=False)

In [3]:
train_df = pd.read_csv('stem_prep_train.csv')
train_df.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,stem_text
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,лет явля клиент эт банк но последн виз прост о...
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,ростовнадон ул ленин час в дан офис не оказа н...
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио,здравств восклицан ран оставля отз ваш банк не...
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...,обраща вам жалоб на незакон списан денежн сред...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,им потребительск кред взят в связьбанк перешед...


In [4]:
test_df = pd.read_csv('stem_prep_test.csv')
test_df.head()

,bank,feeds,date,prep_feeds,cups_num,cups_words,stem_text
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,оформля ипотек в сбербанк подгруж необходим до...
1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19,краткое содержание не рекомендую брать кредит ...,3,в в в,кратк содержан не рекоменд брат кред в эт банк...
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46,добрый день уважаемые сотрудники службы контро...,3,м цб,добр ден уважа сотрудник служб контрол качеств...
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48,обращался получением карты зеленая польза сотр...,0,NaN,обраща получен карт зелен польз сотрудник сооб...
4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41,обратилась в отделение банка на проспекте лени...,9,втб втб втб бесплатно колоссальный боль...,обрат в отделен банк на проспект ленин в отдел...


In [5]:
# Выберем для трейна только те коменты, где есть оценка
TRAIN_1 = train_df[train_df.notna().grades]

In [6]:
TRAIN_2 = train_df[train_df.grades.isna()]

## **TF-IDF**

In [7]:
vect_words = TfidfVectorizer(max_features=81000, analyzer='word', ngram_range=(1, 1))
vect_chars = TfidfVectorizer(max_features=27600, analyzer='char', ngram_range=(1, 3))

In [8]:
all_words_train = vect_words.fit_transform(TRAIN_1.stem_text)
all_chars_train = vect_chars.fit_transform(TRAIN_1.stem_text)

In [9]:
all_words_train.shape

(51476, 65290)

In [10]:
all_chars_train.shape

(51476, 24988)

In [11]:
# Пропуски в train
all_words_train_2 = vect_words.transform(TRAIN_2.stem_text)
all_chars_train_2 = vect_chars.transform(TRAIN_2.stem_text)

In [12]:
all_words_train_2.shape

(23524, 65290)

In [13]:
all_words_test = vect_words.transform(test_df.stem_text)
all_chars_test = vect_chars.transform(test_df.stem_text)

In [14]:
all_words_test.shape

(17220, 65290)

In [15]:
all_chars_test.shape

(17220, 24988)

In [16]:
# Соберем матрицу признаков TF-IDF для теста и трейна
train_feats = sparse.hstack([all_words_train, all_chars_train])
train_2_feats = sparse.hstack([all_words_train_2, all_chars_train_2])
test_feats = sparse.hstack([all_words_test, all_chars_test])

In [17]:
train_feats.shape

(51476, 90278)

# Обучение модели

Возьмем LR с 5 фолдами

In [21]:
model_LR_CV5 = LogisticRegressionCV(solver='saga', cv=5, scoring='f1_micro')

In [22]:
model_LR_CV5.fit(train_feats, TRAIN_1.grades.values)

/Users/aleksandrakozevnikova/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/aleksandrakozevnikova/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/aleksandrakozevnikova/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/aleksandrakozevnikova/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/aleksandrakozevnikova/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did 

LogisticRegressionCV(cv=5, scoring='f1_micro', solver='saga')

In [23]:
# Сохраним модельку
with open('model_tfidf_train_1.pickle', 'wb') as f:
    pickle.dump(model_LR_CV5, f)

with open('model_tfidf_train_1.pickle', 'rb') as f:
    model_LR_CV5 = pickle.load(f)

In [24]:
# Сделаем прогноз
test_predicted = model_LR_CV5.predict(test_feats)

In [25]:
test_predicted = test_predicted.astype(int)

In [26]:
sol = pd.DataFrame({'inds': test_df.index,
                    'grades': test_predicted})
sol

,inds,grades
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
17215,17215,1
17216,17216,1
17217,17217,1
17218,17218,1


In [27]:
sol.to_csv('new_baseline.csv', index=False)